### 🛠️ 1. Initialize notebook variables

Configures everything that's needed for deployment. 

**Modify entries under _1) User-defined parameters_ and _3) Define the APIs and their operations and policies_**.

In [ ]:
import utils
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 1
deployment  = INFRASTRUCTURE.SIMPLE_APIM
tags        = ['azure-maps']       # ENTER DESCRIPTIVE TAG(S)
api_prefix  = 'am-'                            # OPTIONAL: ENTER A PREFIX FOR THE APIS TO REDUCE COLLISION POTENTIAL WITH OTHER SAMPLES
azure_maps_url = 'https://atlas.microsoft.com'  # OPTIONAL: ENTER THE AZURE MAPS URL IF DIFFERENT FROM DEFAULT

# 2) Service-defined parameters (please do not change these)
rg_name = utils.get_infra_rg_name(deployment, index)
supported_infrastructures = [INFRASTRUCTURE.SIMPLE_APIM, INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA]             # ENTER SUPPORTED INFRASTRUCTURES HERE, e.g., [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.AFD_APIM_FE]
utils.validate_infrastructure(deployment, supported_infrastructures)

# 3) Define the APIs and their operations and policies

# Policies
# Named values must be set up a bit differently as they need to have two surrounding curly braces
map_async_geocode_batch_v1_keyauth_post_xml = utils.read_policy_xml('map_async_geocode_batch_v1_keyauth_post.xml')
map_default_route_v2_aad_get_xml = utils.read_policy_xml('map_default_route_v2_aad_get.xml')
map_geocode_v2_aad_get_xml = utils.read_policy_xml('map_geocode_v2_aad_get.xml')

# Map API 
mapApi_v2_default_get = GET_APIOperation2('get-default-route','Get default route','/default/*','This is the default route that will allow all requests to go through to the backend api',map_default_route_v2_aad_get_xml)
mapApi_v1_async_post = APIOperation('async-geocode-batch','Async Geocode Batch','/geocode/batch/async',HTTP_VERB.POST, 'Post geocode batch async endpoint',map_async_geocode_batch_v1_keyauth_post_xml)
mapApi_v2_geocode_get = GET_APIOperation2('get-geocode','Get Geocode','/geocode','Get geocode endpoint',map_geocode_v2_aad_get_xml)
api1 = API('map-api', 'Map API', '/map', 'This is the proxy for Azure Maps', operations=[mapApi_v2_default_get, mapApi_v1_async_post,mapApi_v2_geocode_get, mapApi_v2_foo], tags = tags, serviceUrl=azure_maps_url)

# APIs Array
# apis: List[API] = [api1, apin]
apis: List[API] = [api1]

# 4) Set up the named values, for this specific sample, we are using some of the named values in the API policies defined above that can't be known at this point in the process. For those named values, we are setting them in the main.bicep file.
nvs: List[NamedValue] = [
    NamedValue('azure-maps-arm-api-version','2023-06-01')
]

utils.print_ok('Notebook initialized')

👉🏽 Resource group name      : apim-infra-simple-apim-1 

✅ Notebook initialized ⌚ 12:57:11.067784 


### 🚀 2. Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [24]:
import utils

# 1) Define the Bicep parameters with serialized APIs
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]},
    'namedValues': {'value': [nv.to_dict() for nv in nvs]}
}

# 2) Infrastructure must be in place before samples can be layered on top
if not utils.does_resource_group_exist(rg_name):
    utils.print_error(f'The specified infrastructure resource group and its resources must exist first. Please check that the user-defined parameters above are correctly referencing an existing infrastructure. If it does not yet exist, run the desired infrastructure in the /infra/ folder first.')
    raise SystemExit(1)

# 3) Run the deployment
output = utils.create_bicep_deployment_group(rg_name, rg_location, deployment, bicep_parameters)

# 4) Print a deployment summary, if successful; otherwise, exit with an error
if not output.success:
    raise SystemExit('Deployment failed')

if output.success and output.json_data:
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')

utils.print_ok('Deployment completed')

⚙️ az group show --name apim-infra-simple-apim-1 
⚙️ az group show --name apim-infra-simple-apim-1 
📝 Updated the policy XML in the bicep parameters file 'params.json'
⚙️ az deployment group create --name simple-apim --resource-group apim-infra-simple-apim-1 --template-file main.bicep --parameters params.json --query "properties.outputs" 
👉🏽 APIM API Gateway URL     : https://apim-w6pw4mtuew6ga.azure-api.net 

✅ Deployment completed ⌚ 12:58:55.941252 


### ✅ 3. Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
import utils
from apimrequests import ApimRequests

reqs = ApimRequests(apim_gateway_url)

# 1) Issue a direct request to API Management
reqs.singleGet('/', msg = 'Calling Hello World (Root) API. Expect 200.')

# 2) Issue requests to API Management with Azure Maps APIs
reqs.singleGet('/map/default/geocode?query=15127%20NE%2024th%20Street%20Redmond%20WA', msg = 'Calling Default Route API with SAS Token Auth. Expect 200.')
reqs.singleGet('/map/geocode?query=15127%20NE%2024th%20Street%20Redmond%20WA', msg = 'Calling Geocode v2 API with AAD Auth. Expect 200.')
reqs.singlePostAsync('/map/geocode/batch/async', data={
    "batchItems": [
        {"query": "?query=400 Broad St, Seattle, WA 98109&limit=3"},
        {"query": "?query=One, Microsoft Way, Redmond, WA 98052&limit=3"},
        {"query": "?query=350 5th Ave, New York, NY 10118&limit=1"},
        {"query": "?query=Pike Pl, Seattle, WA 98101&lat=47.610970&lon=-122.342469&radius=1000"},
        {"query": "?query=Champ de Mars, 5 Avenue Anatole France, 75007 Paris, France&limit=1"}
    ]
}, msg = 'Calling Async Geocode Batch v1 API with Share Key Auth. Expect initial 202, then a 200 on the polling response', timeout=120, poll_interval=3)

utils.print_ok('All done!')


ℹ️ Calling Hello World (Root) API. Expect 200. ⌚ 13:21:59.985491 
👉🏽 GET https://apim-w6pw4mtuew6ga.azure-api.net/ 
👉🏽 Response status          : 200 - OK 
👉🏽 Response headers         :
{'Content-Length': '32', 'Date': 'Wed, 18 Jun 2025 18:21:59 GMT', 'Request-Context': 'appId=cid-v1:788e87f2-9135-4c09-896d-02870631a447'} 
👉🏽 Response body            :
Hello World from API Management! 

ℹ️ Calling Default Route API with SAS Token Auth. Expect 200. ⌚ 13:22:00.439404 
👉🏽 GET https://apim-w6pw4mtuew6ga.azure-api.net/map/default/geocode?query=15127%20NE%2024th%20Street%20Redmond%20WA 
👉🏽 Response status          : 200 - OK 
👉🏽 Response headers         :
{'Content-Type': 'application/json; charset=utf-8', 'Date': 'Wed, 18 Jun 2025 18:22:00 GMT', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Vary': 'Accept-Encoding', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'x-ms-azuremaps-region': 'East US', 'X-Content-Type-Options': 'nosniff', 'X-Cache': 'CONFIG_N